In [1]:
import pandas as pd
import numpy as np
import io
import os
import requests
import urllib3
from bs4 import BeautifulSoup as bs4
from importlib import import_module
import sys
sys.path.insert(0,'./support')
sys.path.insert(0,'./wave-site-studies/site_data')

from ipynb.fs.defs.Coordinates import Coordinates
from ipynb.fs.defs.Sites import wave_columns, Site
from ipynb.fs.defs.Grid import Grid,Border
from ipynb.fs.defs.Databases import Database
from ipynb.fs.defs.Periods import Period

In [2]:
def get_Oahu_borders():
    
    url='https://pae-paha.pacioos.hawaii.edu/erddap/info/swan_oahu/index.csv'
    
    db_attributes = pd.read_csv(url)
    db_attributes.set_index(['Attribute Name'], inplace=True)
    
    polygon=db_attributes.loc['geospatial_bounds','Value']
    
    lat_max=db_attributes.loc['geospatial_lat_max','Value']
    lat_min=db_attributes.loc['geospatial_lat_min','Value']
    lon_max=db_attributes.loc['geospatial_lon_max','Value']
    lon_min=db_attributes.loc['geospatial_lon_min','Value']
    
    border_Oahu = Border(float(lat_min),float(lat_max),float(lon_min),float(lon_max))
    
    return border_Oahu

In [3]:
def get_WaveWatch3_borders():
    
    url='https://coastwatch.pfeg.noaa.gov/erddap/info/NWW3_Global_Best/index.csv'
    
    db_attributes = pd.read_csv(url)
    db_attributes.set_index(['Attribute Name'], inplace=True)
    
    polygon=db_attributes.loc['geospatial_bounds','Value']
    
    lat_max=db_attributes.loc['geospatial_lat_max','Value']
    lat_min=db_attributes.loc['geospatial_lat_min','Value']
    lon_max=db_attributes.loc['geospatial_lon_max','Value']
    lon_min=db_attributes.loc['geospatial_lon_min','Value']
    
    border_WW3 = Border(float(lat_min),float(lat_max),float(lon_min),float(lon_max))
    
    return border_WW3

In [4]:
def get_grid_list():

    border_WW3 = get_WaveWatch3_borders()
    WW3 = Grid('WaveWatch3', 'get_WaveWatch3_data', './wave-site-studies/site_data/', Database.WaveWatch3, Period.Tp, border_WW3)

    border_Oahu = get_Oahu_borders()
    Oahu = Grid('Oahu', 'get_Oahu_data', './wave-site-studies/site_data/', Database.Oahu, Period.Tp, border_Oahu)

    #When adding a new grid/region data, the spatial bounds needs to be considered.
    #the list should go from the smaller sized area through the bigger sized one.. 
    #So, an area with the biggest coverage should be at the end of the list.
    # This is done in order to get the data with higher resolution as possible..
    #Alo the key should be the same as the database enum.
    grid_list = {
        'Oahu' : Oahu,
        'WaveWatch3': WW3
    }

    return grid_list

In [2]:
def get_model_data(site, time_start, time_end, swell=False):
    
    grid_list = get_grid_list()
    db=None
    if site.force_db:
        db = grid_list.get(site.force_db.name)
        # coordinates are somethimes displayed differently. Eg. west is not always shown as '-'value. 
        #somethimes its only east like -10 east is 350 east.... Wavewatch3 has this.
        if not (db.border.isin(site.coordinates) or db.border.isin(Coordinates(site.coordinates.latitude, 360+site.coordinates.longitude))):
            print('given site '+site.name+' is not in the border of database '+site.force_db.name+'. Data will come from WaveWatch3 database')
            db = grid_list.get('WaveWatch3')

    else:
        for key,value in grid_list.items(): 
            #map(all(for site in sites),sites)
            if value.border.isin(site.coordinates):
                db = value
                break
    if not db:
         db = grid_list.get('WaveWatch3')
         
    function = db.function
    notebook= db.name+'.ipynb'
    %run $notebook
    site_data = globals()[function](site, time_start, time_end, swell)
    Ttype= db.Ttype
    
    #if no data returned then try wavewatch3 model once...
    if site_data.empty and not db.name=='WaveWatch3':
        print('Site', site.name, 'has no data available. Getting data from a model')
        db = grid_list.get('WaveWatch3')
        function = db.function
        notebook= db.name+'.ipynb'
        %run $notebook
        site_data = globals()[function](site, time_start, time_end, swell)
        Ttype= db.Ttype 

    return site_data, Ttype
